Imports

In [5]:
import sys
import os
from datetime import date

import pandas as pd
import tensorflow as tf
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

print(os.getcwd()+'\\..\\src')
sys.path.append(os.getcwd()+'\\..\\src')
import db_interface

ModuleNotFoundError: No module named 'tensorflow'

Get data

In [4]:
my_db = db_interface.DB_interface("FINANCE_DB")

with my_db:
    df_raw = 

connecting in: FINANCE_DB


Pre Processing

Parameters

Training